In [79]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
from scipy.special import gamma, loggamma, factorial
import scipy.stats
from scipy.interpolate import interp1d
from matplotlib.ticker import AutoMinorLocator 
from matplotlib import rc, font_manager
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from mycolorpy import colorlist as mcp
from matplotlib import legend_handler
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
import collections
import matplotlib
import shapely.geometry
import shapely.ops 
from geopy.geocoders import Nominatim
import geopandas as gpd
import mapclassify
import jenkspy
import contextily as cx
import os
import rioxarray as rx
import pyreadr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
import plots
import seaborn as sns

In [80]:
wd = os.path.dirname(os.getcwd())

In [109]:

country = 'MEX'

if country == 'ARG':
    country_full = 'argentina'
    capital = 'Buenos Aires'
    level = '2'
elif country == 'CHL':
    country_full = 'chile'
    capital = 'Santiago'
    level = '3'
elif country == 'COL':
    country_full = 'colombia'
    level = '2'
elif country == 'MEX':
    country_full = 'mexico'
    level = '2'
elif country == 'PER':
    country_full = 'peru'
    level = '3'

# Admin areas with RDI & population Worldpop

In [110]:
gdf_admin = gpd.read_file(wd + '/data/outputs/' + country + '/gdf_' + country + '_admin_fua_rdi.gpkg')


# Spatial join: admin areas with Facebook population

In [123]:
df_fbpop = pyreadr.read_r('/Volumes/RECAST/data/outputs/' + country_full + '/FB_population/2020_09_pop.rds')
df_fbpop = df_fbpop[None]

# df_fbpop['geometry'] = df_fbpop['geometry'].apply(shapely.wkt.loads)
# gdf_fbpop = gpd.GeoDataFrame(df_fbpop, crs='epsg:4326')

gdf_fbpop = gpd.GeoDataFrame(df_fbpop, geometry=gpd.points_from_xy(df_fbpop.lon, df_fbpop.lat), crs="EPSG:4326"
)


## Admins

In [124]:

gdf_admin_join = gdf_admin

gdf_fbpop_join = gdf_fbpop
gdf_admin_fbpop = gdf_admin_join.sjoin(gdf_fbpop_join, how="left", predicate='intersects')

gdf_admin_fbpop['index'] = gdf_admin_fbpop.index

gdf_group = gdf_admin_fbpop.groupby(['index']).sum().reset_index(drop=True)

gdf_admin_join['n_crisis'] = gdf_group['n_crisis']/(30*3)
gdf_admin_join['n_baseline'] = gdf_group['n_baseline']/(30*3)
gdf_admin_join['perchange'] = (gdf_admin_join['n_crisis']-gdf_admin_join['n_baseline'])/gdf_admin_join['n_baseline']*100
gdf_admin_fbpop = gdf_admin_join


gdf_admin_fbpop['representativiness'] = gdf_admin_fbpop['n_baseline']/gdf_admin_fbpop['pop2020']*100

# gdf_admin_fua_rdi_c_mov.to_file(wd + '/data/outputs/' + country + '/gdf_' + country + '_admin_fua_mov_2022_03.gpkg')


/var/folders/79/65l52xsj7vq_4_t_l6k5bl2c0000gn/T/ipykernel_29621/3895874122.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gdf_group = gdf_admin_fbpop.groupby(['index']).sum().reset_index(drop=True)


KeyError: 'n_crisis'

In [ ]:
plt.scatter(gdf_admin_fbpop['rdi'], gdf_admin_fbpop['representativiness'])

In [ ]:
print([x for x in gdf_admin_fbpop.sort_values(by=['representativiness'], ascending=False)['eFUA_name'][0:30]])

In [ ]:
gdf_admin_fbpop.sort_values(by=['representativiness'], ascending=False)

In [ ]:
gdf_fbpop

In [ ]:
gdf_admin_mov = gpd.read_file(wd + '/data/outputs/MEX/gdf_MEX_admin_fua_mov_2020_04.gpkg')


In [ ]:
gdf_admin_mov